In [9]:
import os

import numpy as np
import pandas as pd

# Linear SVM Experiments

In [10]:
exp_dirs = [
    'no_select_linear_svm',
    'linear_svm_linear',
    'linear_svm_mi',
    'linear_svm_pca',
    'linear_svm_rfe',
    'linear_svm_tree'
]

linear_df = pd.DataFrame()

for d in exp_dirs:
    if d.startswith('no_select'):
        temp = pd.read_parquet(os.path.join(d, 'None_results.parquet'))
        temp.loc[:, 'selection_method'] = 'None'
    else:
        selection_method = d.split('_')[-1]
        if selection_method in ['mi', 'pca', 'rfe', 'sfs']:
            selection_method = selection_method.upper()
        temp = pd.read_parquet(os.path.join(d, f'{selection_method}_results.parquet'))
        temp.loc[:, 'selection_method'] = selection_method
    linear_df = linear_df.append(temp, ignore_index=True)

train_cols = [c for c in linear_df.columns if 'Train' in c]
valid_cols = [c for c in linear_df.columns if 'Valid' in c]
test_cols = [c for c in linear_df.columns if 'Test' in c]

In [13]:
linear_df.groupby(['selection_method']).mean()[test_cols]

,Test Accuracy,Test F1,Test Precision,Test Recall
selection_method,,,,
MI,0.495834,0.492197,0.495399,0.494591
None,0.493861,0.493166,0.495214,0.497898
PCA,0.496097,0.494837,0.497828,0.498602
RFE,0.494364,0.494501,0.495582,0.500225
linear,0.497748,0.423444,0.440206,0.531503
tree,0.496350,0.497392,0.497117,0.503098


# Random Forest Experiments

In [14]:
exp_dirs = [
    'no_select_rf',
    'rf_linear',
    'rf_mi',
    'rf_pca',
    'rf_rfe',
    'rf_tree'
]

rf_df = pd.DataFrame()

for d in exp_dirs:
    if d.startswith('no_select'):
        temp = pd.read_parquet(os.path.join(d, 'None_results.parquet'))
        temp.loc[:, 'selection_method'] = 'None'
    else:
        selection_method = d.split('_')[-1]
        if selection_method in ['mi', 'pca', 'rfe', 'sfs']:
            selection_method = selection_method.upper()
        temp = pd.read_parquet(os.path.join(d, f'{selection_method}_results.parquet'))
        temp.loc[:, 'selection_method'] = selection_method
    rf_df = rf_df.append(temp, ignore_index=True)

train_cols = [c for c in rf_df.columns if 'Train' in c]
valid_cols = [c for c in rf_df.columns if 'Valid' in c]
test_cols = [c for c in rf_df.columns if 'Test' in c]

In [16]:
rf_df.groupby(['selection_method']).mean()[test_cols]

,Test Accuracy,Test F1,Test Precision,Test Recall
selection_method,,,,
MI,0.499273,0.492694,0.500098,0.495520
None,0.492798,0.490871,0.494208,0.499003
PCA,0.496124,0.493844,0.497220,0.501577
RFE,0.493276,0.492542,0.495242,0.500479
linear,0.498626,0.398676,0.410035,0.507568
tree,0.493753,0.489947,0.495533,0.494997
